**Packages**

In [1]:
import numpy as np
import pandas as pd
import math

import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import concatenate
import tifffile

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import os

import scipy
#_____
from numba import cuda 

import os

import tensorflow_addons as tfa
import tensorflow_decision_forests as tfdf

from keras.layers import Flatten, Dense, Dropout

from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer

2023-03-22 03:27:01.993395: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 03:27:02.130074: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-22 03:27:02.130100: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-22 03:27:02.911340: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

**Helper Functions**

In [2]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
   # plt.plot(history.history['accuracy'], label='accuracy')
   # plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.ylim([0, 
            max(
                max(history.history['loss']),
                max(history.history['val_loss'])
            )])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss')
    plt.legend()
    plt.grid(True)

In [3]:
def plot_accuracy(history):
   # plt.plot(history.history['loss'], label='loss')
   # plt.plot(history.history['val_loss'], label='val_loss')
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.ylim([0, 
            max(
                max(history.history['accuracy']),
                max(history.history['val_accuracy'])
            )])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy')
    plt.legend()
    plt.grid(True)

**Data Pre-Processing**

In [4]:
tiles = pd.read_csv(r'./Data/Tiles_binned_zipcode.csv')
tiles.head(5)

Tile_ID       Long2       Lat2       Long1       Lat1    Mid_lat  \
0       36 -122.514446  37.779636 -122.513306  37.778732  37.779184   
1       37 -122.514446  37.778732 -122.513306  37.777829  37.778280   
2      151 -122.513306  37.779636 -122.512166  37.778732  37.779184   
3      152 -122.513306  37.778732 -122.512166  37.777829  37.778280   
4      153 -122.513306  37.777829 -122.512166  37.776925  37.777377   

     Mid_long  Stop_Signs  Paving_historical  Paving_future  ...  94129  \
0 -122.513876         0.0                0.0            0.0  ...      0   
1 -122.513876         0.0                0.0            0.0  ...      0   
2 -122.512736         0.0                0.0            0.0  ...      0   
3 -122.512736         0.0                0.0            0.0  ...      0   
4 -122.512736         0.0                0.0            0.0  ...      0   

   94130  94131 94132  94133  94134  94141  94143  94158  94188  
0      0      0     0      0      0      0      0      0      0  
1      0      0     0      0      0      0      0      0      0  
2      0      0     0      0      0      0      0      0      0  
3      0      0     0      0      0      0      0      0      0  
4      0      0     0      0      0      0      0      0      0  

[5 rows x 53 columns]

In [5]:
tiles.columns

Index(['Tile_ID', 'Long2', 'Lat2', 'Long1', 'Lat1', 'Mid_lat', 'Mid_long',
       'Stop_Signs', 'Paving_historical', 'Paving_future', 'Bus_stop',
       'Collisions_Future', 'Collisions_Historical', 'bin', 'RTTYP_I',
       'RTTYP_M', 'RTTYP_O', 'RTTYP_S', 'RTTYP_U', 'Collisions_Future_binary',
       'Collisions_Historical_binary', 'bins_numeric', 'zip_code', '94101',
       '94102', '94104', '94105', '94107', '94108', '94109', '94110', '94111',
       '94112', '94114', '94115', '94116', '94117', '94118', '94121', '94122',
       '94123', '94124', '94127', '94129', '94130', '94131', '94132', '94133',
       '94134', '94141', '94143', '94158', '94188'],
      dtype='object')

Split into training and test

In [6]:
x_train, x_test, y_train, y_test = train_test_split(tiles[['Tile_ID', 'Long2', 'Lat2', 'Long1', 'Lat1', 'Mid_lat', 'Mid_long',
       'Stop_Signs', 'Paving_historical', 'Paving_future', 'Bus_stop',
       'Collisions_Future', 'Collisions_Historical', 'RTTYP_I',
       'RTTYP_M', 'RTTYP_O', 'RTTYP_S', 'RTTYP_U', 'Collisions_Future_binary',
       'Collisions_Historical_binary', 'bins_numeric', 'zip_code', '94101',
       '94102', '94104', '94105', '94107', '94108', '94109', '94110', '94111',
       '94112', '94114', '94115', '94116', '94117', '94118', '94121', '94122',
       '94123', '94124', '94127', '94129', '94130', '94131', '94132', '94133',
       '94134', '94141', '94143', '94158', '94188']], 
                                   tiles['bin'],
                                   random_state=104, 
                                   test_size=0.20, 
                                   shuffle=True)

Image Pre-Processing

In [7]:
IMAGE_PATH = './Satellite Imagery/Satellite Images Tiled/' 

def preprocess_data_part1(IMAGE_PATH):
    """ Generate lists of images and labelsbased on temp_no_refer and temp_refer lists
    
    Params:
    -------
    IMAGE_PATH (str): path to directory with images.
    
    Returns:
    --------
    images_mini  (np.ndarray): Images of shape (N, 149 3)
    """
    
    data_mini = []
    data_mini_test = []
    for id in x_train['Tile_ID']:    
                    
        # read image and store as matrix            
        # Index at the end makes all images the same size (they sometimes differ by 1 pixel)
        image = tifffile.imread(IMAGE_PATH + str(id) + '.tif')[0:148, 0:188, :]
        
        #grayscale
        for i in image:
            for j in i:
                data_mini.append([np.mean(j[0:3]), j[3]])
        
        # append to images
        #data_mini.append(image)
 
    # stack images and trasnform to array
    images_mini = np.stack(data_mini)
    
    for id in x_test['Tile_ID']:    
                    
        # read image and store as matrix            
        # Index at the end makes all images the same size (they sometimes differ by 1 pixel)
        image = tifffile.imread(IMAGE_PATH + str(id) + '.tif')[0:148, 0:188, :]
        
        #grayscale
        for i in image:
            for j in i:
                data_mini_test.append([np.mean(j[0:3]), j[3]])
        
        # append to images
        #data_mini_test.append(image)
 
    # stack images and trasnform to array
    images_mini_test = np.stack(data_mini_test)
    
    return images_mini, images_mini_test

In [ ]:
images_mini, images_mini_test = preprocess_data_part1(IMAGE_PATH)
print('train ', np.shape(images_mini))
print('test ', np.shape(image_mini_test))

Street Data Pre-Processing

In [ ]:
street = np.asarray(x_train[['Tile_ID', 'Long2', 'Lat2', 'Long1', 'Lat1', 'Mid_lat', 'Mid_long',
       'Stop_Signs', 'Paving_historical', 'Paving_future', 'Bus_stop',
       'Collisions_Future', 'Collisions_Historical', 'RTTYP_I', 'RTTYP_M',
       'RTTYP_O', 'RTTYP_S', 'RTTYP_U', 'Collisions_Future_binary',
       'Collisions_Historical_binary', 'bins_numeric', 'zip_code', '94101',
       '94102', '94104', '94105', '94107', '94108', '94109', '94110', '94111',
       '94112', '94114', '94115', '94116', '94117', '94118', '94121', '94122',
       '94123', '94124', '94127', '94129', '94130', '94131', '94132', '94133',
       '94134', '94141', '94143', '94158', '94188']]).astype('float32')
street_mini = []
for row in range(len(street)):
    street_mini.append([[street[row]]])
street_mini = np.stack(street_mini)
print('train ', np.shape(street_mini))

street_test = np.asarray(x_test[['Tile_ID', 'Long2', 'Lat2', 'Long1', 'Lat1', 'Mid_lat', 'Mid_long',
       'Stop_Signs', 'Paving_historical', 'Paving_future', 'Bus_stop',
       'Collisions_Future', 'Collisions_Historical', 'RTTYP_I', 'RTTYP_M',
       'RTTYP_O', 'RTTYP_S', 'RTTYP_U', 'Collisions_Future_binary',
       'Collisions_Historical_binary', 'bins_numeric', 'zip_code', '94101',
       '94102', '94104', '94105', '94107', '94108', '94109', '94110', '94111',
       '94112', '94114', '94115', '94116', '94117', '94118', '94121', '94122',
       '94123', '94124', '94127', '94129', '94130', '94131', '94132', '94133',
       '94134', '94141', '94143', '94158', '94188']]).astype('float32')
street_mini_test = []
for row in range(len(street_test)):
    street_mini_test.append([[street_test[row]]])
street_mini_test = np.stack(street_mini_test)
print('test ',np.shape(street_mini_test))

In [ ]:
input_image_street = np.hstack(
    (street_mini.reshape((8376,52)),
     images_mini.reshape(8376,148*188*4))
)
np.shape(input_image_street)

In [ ]:
input_image_street_test = np.hstack(
    (street_mini_test.reshape((2095,52)),
     images_mini_test.reshape(2095,148*188*4))
)
np.shape(input_image_street_test)

**Noriel's Models**

In [ ]:
y_train[0:5]

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)

In [ ]:
y_train[0:5]

In [ ]:
# from sklearn import preprocessing
# scaler = preprocessing.StandardScaler().fit(input_image_street)
# X_scaled = scaler.transform(input_image_street)
# X_scaled

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# # Create the parameter grid based on the results of random search 
# param_grid = {
#     'bootstrap': [True],'max_depth': [20,30,40, 100, 110],
#     'max_features': ['sqrt'],'min_samples_leaf': [5,10,15],
#     'min_samples_split': [40,50,60], 'n_estimators': [150, 200, 250]
# }
# # Create a based model
# rf = RandomForestClassifier()
# # Instantiate the grid search model
# grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
#                           cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
# grid_search.fit(X_scaled, y_train)

In [ ]:
# grid_search.best_params_

In [ ]:
rf_model = tfdf.keras.RandomForestModel(categorical_algorithm = 'CART', num_trees=300, max_depth = 110)

rf_model.fit(input_image_street,
    y_train,    
    epochs=1,
    verbose=1)

In [ ]:
print(rf_model.summary())

In [ ]:
##uncomment to show training log
#rf_model_train.make_inspector().training_logs()
rf_model.make_inspector().evaluation()

In [ ]:
import matplotlib.pyplot as plt

logs = rf_model.make_inspector().training_logs()

plt.plot([log.evaluation.num_examples for log in logs], [log.evaluation.loss for log in logs], label="training data")
plt.xlabel("Number of trees")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

logs = rf_model.make_inspector().training_logs()

plt.plot([log.evaluation.num_examples for log in logs], [log.evaluation.accuracy for log in logs], label="training data")
plt.xlabel("Number of trees")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
#evaluate the model
rf_model.evaluate(input_image_street_test)

In [ ]:
rf_model.predict(input_image_street_test)

In [ ]:
predicted_result = rf_model.predict(input_image_street_test)
test_values = []
for i in predicted_result: 
    input_list = i
    max_value = max(input_list)
    index = [index for index, item in enumerate(input_list) if item == max_value]
    test_values.append(index[0])
print('macro f1: ', f1_score(y_test, test_values, average = 'macro' ))
print('f1 by class: ', f1_score(y_test, test_values, average = None ))

In [ ]:
# metric = tfa.metrics.F1Score(num_classes=3, threshold=0.5)
# y_true = np.array([[1, 1, 1],
#                    [1, 0, 0],
#                    [1, 1, 0]], np.int32)
# y_pred = np.array([[0.2, 0.6, 0.7],
#                    [0.2, 0.6, 0.6],
#                    [0.6, 0.8, 0.0]], np.float32)
# metric.update_state(y_true, y_pred)
# result = metric.result()
# result.numpy()


**Gradient Boosted Tree**

In [ ]:
gbt_model = tfdf.keras.GradientBoostedTreesModel(categorical_algorithm = 'CART')

gbt_model.fit(input_image_street,
    y_train,
    #validation_data=[x_test[['Collisions_Historical', 'Mid_lat','Mid_long', 'Stop_Signs', 'Paving_historical', 'Bus_stop']], y_test],     
    epochs=1,
    verbose=1)

In [ ]:
print(gbt_model.summary())

In [ ]:
##uncomment to show training log
#gbt_model.make_inspector().training_logs()
gbt_model.make_inspector().evaluation()

In [ ]:
import matplotlib.pyplot as plt

logs = gbt_model.make_inspector().training_logs()

plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs], label="training data")
plt.xlabel("Number of trees")
plt.ylabel("Loss")
plt.legend()

plt.show()

In [ ]:
gbt_model.evaluate(x_test)

Random Forest With Neural Network

In [ ]:
# x: 'Collisions_Historical', 'Mid_lat','Mid_long', 'Stop_Signs', 'Paving_historical', 'Bus_stop','RTTYP_I','RTTYP_M','RTTYP_O','RTTYP_S','RTTYP_U'
# y: 'Collisions_Future'

def create_linear_model():
    # DENSE LAYERS 
    input = tf.keras.layers.Input(shape=(52,)) # (111307,))

    outputs = layers.Dense(1)(input)

    linear_model = tf.keras.models.Model(inputs=input, outputs=outputs, name="linear_model")

    linear_model.summary()

    linear_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mean_squared_error'
        )
    
    return linear_model

linear_model = create_linear_model()

history = linear_model.fit(
    input_image_street,
    y_train,
    epochs=25,
    # Suppress logging.
     verbose=1,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

In [ ]:
plot_loss(history)

In [ ]:
np.shape(x_test)

In [ ]:
predicted_result = linear_model.predict(x_test)
test_values = []
for i in predicted_result: 
    input_list = i
    max_value = max(input_list)
    index = [index for index, item in enumerate(input_list) if item == max_value]
    test_values.append(index[0])
print('macro f1: ', f1_score(y_test, test_values, average = 'macro' ))
print('f1 by class: ', f1_score(y_test, test_values, average = None ))

**Logistic Regression**

In [ ]:
# x: 'Collisions_Historical', 'Mid_lat','Mid_long', 'Stop_Signs', 'Paving_historical', 'Bus_stop','RTTYP_I','RTTYP_M','RTTYP_O','RTTYP_S','RTTYP_U'
# y: 'Collisions_Future'

def create_logistic_model():
    # DENSE LAYERS 
    input = tf.keras.layers.Input(shape=(53,))
    
    # input = keras.layers.Flatten()(input)

    outputs = keras.layers.Dense(units = 1, activation = 'sigmoid')(input)

    log_model = tf.keras.models.Model(inputs=input, outputs=outputs, name="logistic_model")

    log_model.summary()

    log_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="bce",
        metrics=['accuracy']
        )
    
    return log_model

logistic_model = create_logistic_model()

class_weight = {0: 1,
                1: 100000,
                2: 100000,
                3: 100000,
                4: 100000,
                5: 100000,
                6: 100000,
                7: 100000,
                8: 100000,
                9: 100000,
                10: 100000,
                #11: 100000,
               }

history = logistic_model.fit(
    input_image_street,
    y_train,
     # y_train_2,
    epochs=50,
    # Suppress logging.
     # verbose=1,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2,
    # class_weight = class_weight
    )

In [ ]:
plot_loss(history)